This notebook can be run on Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ZKC19940412/water_ice_nep/colab-examples/example-4.ipynb)

In Colab, you can enable the GPU acceleration from `Edit` > `Notebook Settings` > `Hardware accelerator` > `GPU`.

# Install TDMDpy from Source ($\sim$ 2 min)

In [ ]:
%%capture
%cd /content/
# Fetch tdmdpy repo
! git clone https://github.com/ZKC19940412/tdmdpy.git

# Install tdmdpy from source
%cd tdmdpy
! pip install .

# Install [GPUMD](https://github.com/brucefan1983/GPUMD) from Source ($\sim$ 2 min)
- More instructions can be found : https://gpumd.org/installation.html

In [ ]:
%%capture
%cd /content/
! git clone https://github.com/brucefan1983/GPUMD.git
%cd /content/GPUMD/src/
! make -j 8
! echo "GPUMD installation finishes!"
%cd /content/

# Clean up Workspace and Fetch Nessary Files

In [ ]:
%cd /content/
! git clone https://github.com/ZKC19940412/water_ice_nep.git
! rm -r sample_data

# Import Necessary Packages

In [ ]:
import mdtraj as mdt
import numpy as np
from pynvml import *
from pylab import *
import scipy.constants as spc
from tdmdpy.atom_manipulate import decompose_dump_xyz
from tdmdpy.atom_manipulate import load_with_cell
from tdmdpy.create_systems import generate_water_box
from tdmdpy.scorer import score_property
from tdmdpy.thermodynamic_properties import get_block_average_quantities

# Custom Function Define

In [ ]:
def set_fig_properties(ax_list):
    tl = 6
    tw = 2
    tlm = 4

    for ax in ax_list:
        ax.tick_params(which='major', length=tl, width=tw)
        ax.tick_params(which='minor', length=tlm, width=tw)
        ax.tick_params(which='both', axis='both',
                       direction='in', right=True, top=True,
                       left=True)

# Obtain Information about GPU Architecture for Particular colab Instance

In [ ]:
if __name__ == "__main__":

    # Initialize nvml object
    nvmlInit()
    print("Driver Version:", nvmlSystemGetDriverVersion())
    deviceCount = nvmlDeviceGetCount()

    # Loop through all avaliable devices
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        print("Device", i, ":", nvmlDeviceGetName(handle))

# Perform Simulations to Compute  Isothermal Compressibility ($\kappa_{T}$)

## Create Liquid Water in Cubic Box

In [ ]:
if __name__ == "__main__":
    generate_water_box(target_density = 0.994,
                       number_of_molecules=512,
                       is_pre_equilibrate=False)

## Compose run.in

In [ ]:
%%writefile run.in
# NEP potential
potential /content/water_ice_nep/nep-pre-train-model/nep.txt

# time step for 0.5 fs
time_step 0.5

# Initialize velocity at 298K
velocity 298

# Run NPT equilibration with SCR method for 2598K Tini and Tend,
# and 100 for Tcoupling,  1.01325 bar for pressures, and
# 2 Gpa for pressure coupling and 1000 steps
ensemble npt_scr 298 298 100 0.000101325 2 1000

# run 8000 steps, equal to 4 ps simulation
run 8000

# Run NPT production with SCR method for 2598K Tini and Tend,
# and 100 for Tcoupling,  1.01325 bar for pressures, and
# 2 Gpa for pressure coupling and 1000 steps
ensemble npt_scr 298 298 100 0.000101325 2 1000

# dump extended xyz with every 100 steps, dump force and velocity too
dump_exyz 100 1 1

# dump themodynamic quantities every 100 steps
dump_thermo 100

# Run 100000 steps, equal to 50 ps simulation
run 100000

## Peform Simualtions ($\sim$ 15 min)
- 8000 and 100000 steps used here only serve as illustration purpose.

In [ ]:
! /content/GPUMD/src/gpumd < run.in

## Compute Mass Density ($\rho$)

In [ ]:
if __name__ == "__main__":

    # Load in temperature data from thermo.out
    data = np.loadtxt('thermo.out')

    # Extract box dimension from thermo.out
    # length scale goes from angstrom to nm
    cell_length_matrix = data[:, -3:] / 10.0
    cell_angle_matrix = 90 * np.ones_like(cell_length_matrix)

    # Decompose dump.xyz
    decompose_dump_xyz('dump.xyz')

    # Inject Reference PDB file into the trajectory'
    pos_trajectory = load_with_cell('pos.xyz', cell_length_matrix,
                                    cell_angle_matrix,
                                    top='ini_pos.pdb')

    # Compute mass density
    mass_density = mdt.density(pos_trajectory)
    mass_density_in_block = get_block_average_quantities(mass_density,n_block=5)
    block_average_mass_density =  mass_density_in_block.mean()/1000.0

## Visualize $\rho$ and T vs Time from Production Run

In [ ]:
if __name__ == "__main__":

    #  Set up Figure Styles
    aw = 2
    fs = 22
    font = {'size': fs}
    matplotlib.rc('font', **font)
    matplotlib.rc('axes', linewidth=aw)

    # Denote time step and sample rate
    time_step = 5e-4
    sample_rate = 100

    # Derive time span
    time_span = time_step * sample_rate * np.arange(0, len(data), 1)

    # Load temperature and compute its block averages
    temperature = data[:, 0]
    block_average_temperature = get_block_average_quantities(temperature,
                                                             n_block=5)

    print('Inspect Properties from NPT Simulations: ')
    print('\n')
    figure(figsize=(8, 6))
    set_fig_properties([gca()])
    plot(time_span, temperature, label='T')
    plot(time_span, np.round(
        block_average_temperature.mean(), 3) * np.ones_like(
        time_span), 'r--',
         label=r'$T_{block \ avg}$', lw=3)
    xlabel('Time (ps)')
    ylabel('Temperature (K)')
    ylim([260, 320])
    legend(loc=4, fontsize=16)
    show()

    print('\n')
    figure(figsize=(8, 6))
    set_fig_properties([gca()])
    plot(time_span, mass_density/1000.0, label=r'$\rho$')
    plot(time_span,
         np.round(block_average_mass_density,3) *
         np.ones_like(time_span), 'r--',
         label=r'$\rho_{block \ avg}$', lw=3)
    xlabel('Time (ps)')
    ylabel(r'$\rho \ (g/cm^{3})$')
    ylim([0.96, 1.05])
    legend(loc=4, fontsize=16)
    show()

# Compute isothermal compressibility ($\kappa_{T}$)
 - $\kappa_{T} = \frac{<\sigma_{V}^{2}>}{<V> k_{B}T} = \frac{<V^{2}> -<V>^{2}}{<V> k_{B}T}$
 - Epxerimental reference : 45.3 Mbar$^{-1}$

In [ ]:
if __name__ == "__main__":

    # Load in temperature data from thermo.out
    data = np.loadtxt('thermo.out')

    # Denote number of blokcs
    n_blocks = 5

    # Extract box lengths in angstrom
    Lx = np.split(data[:, -3],n_blocks)
    Ly = np.split(data[:, -2], n_blocks)
    Lz = np.split(data[:,-1], n_blocks)

    # Preset empty zero arrays
    cov_V_block_averages = np.zeros([n_blocks])
    V_mean_block_averages = np.zeros([n_blocks])

    # Compute quantities for each block
    for i in range(n_blocks):
        V = (Lx[i] * Ly[i] * Lz[i]) * (spc.angstrom ** 3)
        V_mean_block_averages[i] = np.mean(V)
        cov_V_block_averages [i] = np.cov(V)

    T = get_block_average_quantities(data[:, 0], n_block=n_blocks)
    kBT = spc.value('Boltzmann constant') * T
    kappa_T = spc.giga * spc.hecto * cov_V_block_averages/(
        V_mean_block_averages * kBT)
    print('kappa_T: %.1f ± %.1f Mbar^-1' %(kappa_T.mean(),(kappa_T.std()/len(kappa_T) )))